### Imports

In [ ]:
# To resolve mkl bug when importing skimage (Seattle Computer Only) 
import mkl 
mkl.get_max_threads()

In [2]:
import tensorflow as tf

ImportError: Could not find 'cudart64_80.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 8.0 from this URL: https://developer.nvidia.com/cuda-toolkit

In [3]:
from keras.layers import Input, Dense

Using TensorFlow backend.


ImportError: Could not find 'cudart64_80.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 8.0 from this URL: https://developer.nvidia.com/cuda-toolkit

In [ ]:
from keras.models import Model

In [ ]:
import numpy as np

In [ ]:
import os

In [ ]:
import cv2

In [ ]:
import matplotlib.pyplot as plt

### Paths to training, validating and testing data

In [ ]:
path_train_img = "F:\\Apar\\Study\\Northeastern\\Semester1\\PatternRecognitionAndCV\\Assignments\\Assignment_2\\Solution\\BSR\\BSDS500\\data\\images\\train"
path_val_img = "F:\\Apar\\Study\\Northeastern\\Semester1\\PatternRecognitionAndCV\\Assignments\\Assignment_2\\Solution\\BSR\\BSDS500\\data\\images\\val"
path_test_img = "F:\\Apar\\Study\\Northeastern\\Semester1\\PatternRecognitionAndCV\\Assignments\\Assignment_2\\Solution\\BSR\\BSDS500\\data\\images\\test"

## Patches

### Method for Sliding window

In [ ]:
def sliding_window(image, window_shape, step_size):
    rows = image.shape[0]
    cols = image.shape[1]
    for x in range(0, rows, step_size):
        for y in range(0, cols, step_size):
            yield(x, y, image[x:x+window_shape[0], y:y+window_shape[1]])
            

### Method for Creating patches

In [ ]:
def create_clean_patches(path_images, window_shape, step_size):
    patches = []
    print ('Creating CLEAN ' + str(window_shape[0]) + '*' + str(window_shape[1]) + ' patches for images in ' + path_images)
    for i, filename in enumerate(os.listdir(path_images)):
        image = cv2.imread(path_images + os.path.sep + filename, cv2.IMREAD_GRAYSCALE)
        for (x, y, window) in sliding_window(image, window_shape, step_size):
            if(window.shape == window_shape):
                patches.append(window.flatten())
    print ('All Patched Created added to list')
    print()
    return np.array(patches)

In [ ]:
#Citation: http://www.magikcode.com/?p=240
def add_gaussian_noise(image_in, noise_sigma):
    temp_image = np.float64(np.copy(image_in))

    h = temp_image.shape[0]
    w = temp_image.shape[1]
    np.random.seed(123456789)
    noise = np.random.randn(h, w) * noise_sigma

    noisy_image = np.zeros(temp_image.shape, np.float64)
    if len(temp_image.shape) == 2:
        noisy_image = temp_image + noise
    else:
        noisy_image[:,:,0] = temp_image[:,:,0] + noise
        noisy_image[:,:,1] = temp_image[:,:,1] + noise
        noisy_image[:,:,2] = temp_image[:,:,2] + noise

    return noisy_image

In [ ]:
def create_noisy_patches(path_images, window_shape, step_size):
    patches = []
    print ('Creating NOISY ' + str(window_shape[0]) + '*' + str(window_shape[1]) + ' patches for images in ' + path_images)
    for i, filename in enumerate(os.listdir(path_images)):
        image = cv2.imread(path_images + os.path.sep + filename, cv2.IMREAD_GRAYSCALE)
        #add noise to image
        image = add_gaussian_noise(image, 25)
        for (x, y, window) in sliding_window(image, window_shape, step_size):
            if(window.shape == window_shape):
                patches.append(window.flatten())
    print ('All Patched Created added to list')
    print()
    return np.array(patches)

### Creating Patches

In [ ]:
window_shape = (8,8)
step_size = 8

#### Clean Patches

In [ ]:
clean_patches_train_8 = create_clean_patches(path_train_img, window_shape, step_size)
clean_patches_val_8 = create_clean_patches(path_val_img, window_shape, step_size)
clean_patches_test_8 = create_clean_patches(path_test_img, window_shape, step_size)

#### Noisy Patches

In [ ]:
noisy_patches_train_8 = create_noisy_patches(path_train_img, window_shape, step_size)
noisy_patches_val_8 = create_noisy_patches(path_val_img, window_shape, step_size)
noisy_patches_test_8 = create_noisy_patches(path_test_img, window_shape, step_size)

#### Checking patches shape

In [ ]:
print (clean_patches_train_8.shape)
print (clean_patches_val_8.shape)
print (clean_patches_test_8.shape)

print (noisy_patches_train_8.shape)
print (noisy_patches_val_8.shape)
print (noisy_patches_test_8.shape)


#### Normalizing patches

In [ ]:
clean_patches_train_8 = clean_patches_train_8.astype('float32') / 255.
clean_patches_val_8 = clean_patches_val_8.astype('float32') / 255.
clean_patches_test_8 = clean_patches_test_8.astype('float32') / 255.

noisy_patches_train_8 = noisy_patches_train_8.astype('float32') / 255.
noisy_patches_val_8 = noisy_patches_val_8.astype('float32') / 255.
noisy_patches_test_8 = noisy_patches_test_8.astype('float32') / 255.


## Autoencoder - 1

#### Dimensions

In [ ]:
input_dim_1 = 8*8
encoding_dim_1 = 16*16
decoding_dim_1 = input_dim_1

#### Single fully connected layer

In [ ]:
input_img_1 = Input(shape=(input_dim_1,))
encoded_1 = Dense(encoding_dim_1, activation='relu')(input_img_1)
decoded_1 = Dense(decoding_dim_1, activation='sigmoid')(encoded_1)

autoencoder_1 = Model(input_img_1, decoded_1)

#### Encoder model

In [ ]:
encoder_1 = Model(input_img_1, encoded_1)

#### Configuring Autoencoder

In [ ]:
autoencoder_1.compile(optimizer='adadelta', loss='mean_squared_error')

#### Training Autoencoder

In [ ]:
history_1 = autoencoder_1.fit(
    clean_patches_train_8, clean_patches_train_8,
    epochs = 50,
    batch_size = 256,
    shuffle = True,
    validation_data = (clean_patches_val_8, clean_patches_val_8))

#### Plotting the loss

In [ ]:
print (history_1.history.keys())

plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### Output of encoding layer

In [ ]:
encoded_imgs_1 = encoder_1.predict(clean_patches_test_8)
print (encoded_imgs_1.shape)

## Autoencoder - 2

#### Dimensions

In [ ]:
input_dim_2 = 16*16
encoding_dim_2 = 32*32
decoding_dim_2 = input_dim_2

#### Single fully connected layer

In [ ]:
input_img_2 = Input(shape=(input_dim_2,))
encoded_2 = Dense(encoding_dim_2, activation='relu')(input_img_2)
decoded_2 = Dense(decoding_dim_2, activation='sigmoid')(encoded_2)

autoencoder_2 = Model(input_img_2, decoded_2)

#### Encoder model

In [ ]:
encoder_2 = Model(input_img_2, encoded_2)

#### Configuring Autoencoder

In [ ]:
autoencoder_2.compile(optimizer='adadelta', loss='mean_squared_error')

#### Dividing data into training and validation

In [ ]:
encoded_imgs_length = encoded_imgs_1.shape[0]
one_third = int(encoded_imgs_length / 3)
train_data_2 = encoded_imgs_1[:2*one_third]
val_data_2 = encoded_imgs_1[2*one_third:]
print(train_data_2.shape)
print(val_data_2.shape)

#### Training Autoencoder

In [ ]:
history_2 = autoencoder_2.fit(
    train_data_2, train_data_2,
    epochs = 50,
    batch_size = 256,
    shuffle = True,
    validation_data = (val_data_2, val_data_2))

#### Plotting the loss

In [ ]:
print (history_2.history.keys())

plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Stacked Sparse Denoising Autoencoder

#### Dimensions

In [ ]:
input_dim = 8*8
encoding_dim1 = 16*16
encoding_dim2 = 32*32
decoding_dim1 = encoding_dim1
decoding_dim2 = input_dim

#### Stack of layers

In [ ]:
input_img = Input(shape=(input_dim,))

encoded1 = Dense(encoding_dim1, activation='relu')(input_img)
encoded2 = Dense(encoding_dim2, activation='relu')(encoded1)

decoded1 = Dense(decoding_dim1, activation='sigmoid')(encoded2)
decoded2 = Dense(decoding_dim2, activation='sigmoid')(decoded1)

autoencoder = Model(input_img, decoded2)

#### Encoder model

In [ ]:
encoder1 = Model(input_img, encoded1)

encoded_img1 = Input(shape=(encoding_dim1,))
encoded2_layer = autoencoder.layers[-3]
encoder2 = Model(encoded_img1, encoded2_layer(encoded_img1))

encoder1.set_weights(encoder_1.get_weights())
encoder2.set_weights(encoder_2.get_weights())

#### Configuring Autoencoder

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

#### Training Autoencoder

In [ ]:
history = autoencoder.fit(
    noisy_patches_train_8, clean_patches_train_8,
    epochs = 50,
    batch_size = 256,
    shuffle = True,
    validation_data = (noisy_patches_val_8, clean_patches_val_8))

#### Plotting the loss

In [ ]:
print (history.history.keys())

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Saving weights

In [ ]:
autoencoder.save_weights("SSDA_weights.h5")

### Testing Data

In [ ]:
denoised_patches_test_8 = autoencoder.predict(noisy_patches_test_8)

### PSNR

In [ ]:
import math

In [ ]:
def calculate_mse(image1, image2):
    squared_error = (image1-image2)**2
    sum = np.sum(squared_error)
    mse = sum / float(squared_error.size)    
    return mse

In [ ]:
MAX = 1 #We normalised the data
def calculate_psnr(image1, image2):
    return 20*math.log10(MAX) - 10*math.log10(calculate_mse(image1, image2))

In [ ]:
def list_psnr(image_list_1, image_list_2):
    list_psnr= []
    for i in range(1000):
        psnr = calculate_psnr(image_list_1[i], image_list_2[i])
        list_psnr.append(psnr)
    return list_psnr

In [ ]:
psnr_noisy_clean = list_psnr(noisy_patches_test_8, clean_patches_test_8)
psnr_denoised_clean = list_psnr(denoised_patches_test_8, clean_patches_test_8)

In [ ]:
def plot_psnr(list1, list2):
    plt.plot(list1)
    plt.plot(list2)
    plt.title('PSNRs')
    plt.ylabel('psnr')
    plt.xlabel('patches')
    plt.legend(['noisy-clean', 'denoised-clean'], loc='upper left')
    plt.show()

In [ ]:
plot_psnr(psnr_noisy_clean, psnr_denoised_clean)